In [1]:
# read data
import numpy as np
import pandas as pd
import string 
from gmplot import gmplot
import googlemaps

gmaps = googlemaps.Client(key='___key_removed___')

print ("Reading data...")
df = pd.read_csv('crime.txt')
df.columns = ['agency', 'crime', 'time', 'addr', 'zip', 'community']
df['crime'] = df['zip'].astype(str)

Reading data...


In [2]:

#check whether the lat, lon is near San Diego, can shrink this to ((32, 34), (-118, -115))
def check_CA_lat_lon(lat, lon):
    if(lat >= 32 and lat <= 35) and (lon>= -118 and lon <= -115):
        return True
    else:
        return False
    
def get_last_week_crime_data(date, df):
    from datetime import datetime
    addr = []
    addr_dict = {}
    for i in range(len(df['time'])):
        if(len(df['time'][i]) == 0 or len(df['addr'][i]) == 0):
            continue
        dif = (datetime.strptime(df['time'][i], "%m/%d/%Y %H:%M:%S") - datetime.strptime(date, "%m/%d/%Y %H:%M:%S")).days
        if dif >= -7 and dif <0:
            if df['addr'][i] in addr_dict:
                addr_dict[df['addr'][i]] += 1
            else:
                addr_dict[df['addr'][i]] = 1
            addr.append(df['addr'][i])
    print("all addresses: ", len(addr))
    print("all unique address: ", len(addr_dict))
    #print(addr_dict)
    res_list = []
    for key in addr_dict:
        geocode_result = gmaps.geocode(key)
        if(len(geocode_result) == 0):
            continue
        curr_lat = geocode_result[0]["geometry"]["location"]['lat']
        curr_lon = geocode_result[0]["geometry"]["location"]['lng']
        if(check_CA_lat_lon(curr_lat, curr_lon)):
            print(curr_lat, curr_lon, key)
            latlon = (curr_lat, curr_lon)
            for i in range(addr_dict[key]):
                res_list.append(latlon)
    print("all valid near SD crime number in the past week", len(res_list))
    return res_list

In [3]:
date = '2/5/2018 16:30:00'
crime_lat_lon_list = get_last_week_crime_data(date, df)


('all addresses: ', 2368)
('all unique address: ', 1868)
(33.1951409, -117.3829995, '200  BLOCK PIER VIEW WAY')
(33.2235143, -117.2629365, '1500 N  BLOCK MELROSE DRIVE')
(32.7703275, -116.9641387, '10800  BLOCK MELVA ROAD')
(32.786744, -116.8327492, '5400  BLOCK CASINO WAY')
(33.1813236, -117.3473247, '2100 E  BLOCK VISTA WAY')
(32.9816305, -117.062829, '14700  BLOCK POMERADO ROAD')
(33.2430162, -117.2890012, '600  BLOCK TANGIER COURT')
(32.8707552, -117.2160347, '8800  BLOCK COSTA VERDE BOULEVARD')
(32.7634708, -117.1071488, '4100  BLOCK ADAMS AVENUE')
(32.7406373, -117.0263422, 'BROADWAY KEMPF STREET')
(32.74657, -117.252083, '5000  BLOCK NIAGARA AVENUE')
(33.0224475, -117.2833454, '2000  BLOCK SAN ELIJO AVENUE')
(33.1245522, -117.2791816, 'BONAIR ROAD / PALOMAR PLACE')
(32.8146055, -117.2164107, '4000  BLOCK HUERFANO AVENUE')
(32.7780206, -116.8401778, '5500  BLOCK C SYCAUN ROAD')
(32.7415378, -117.2423947, '1900  BLOCK GUIZOT STREET')
(32.7356288, -117.1250407, '3200  BLOCK PALM ST

In [48]:
def find_max_inten(date, df):
    addr = []
    addr_dict = {}
    for i in range(len(df['time'])):
        if(len(df['time'][i]) == 0 or len(df['addr'][i]) == 0):
            continue
        dif = (datetime.strptime(df['time'][i], "%m/%d/%Y %H:%M:%S") - datetime.strptime(date, "%m/%d/%Y %H:%M:%S")).days
        if dif >= -7 and dif <0:
            if df['addr'][i] in addr_dict:
                addr_dict[df['addr'][i]] += 1
            else:
                addr_dict[df['addr'][i]] = 1
            addr.append(df['addr'][i])
    print("all addresses: ", len(addr))
    print("all unique address: ", len(addr_dict))
    #print(addr_dict)
    res_list = []
    max_inten = 0
    max_dir = ""
    for key in addr_dict:
        if key == '0  BLOCK GBDF':
            continue
        if addr_dict[key] > max_inten:
            max_inten = addr_dict[key]
            max_dir = key
    print("max_inten is : ", max_inten)
    print("max_inten address is : ", max_dir)
find_max_inten(date, df)

('all addresses: ', 2368)
('all unique address: ', 1868)
('max_inten is : ', 9)
('max_inten address is : ', '5500  BLOCK GROSSMONT CENTER DRIVE')


In [86]:
def generate_past_week_html(crime_lat_lon_list, center_lat = 32.7269669, center_lon = -117.1647094): 
    from gmplot import gmplot
    gmap = gmplot.GoogleMapPlotter(center_lat, center_lon, 13)
    gmap.marker(center_lat, center_lon, 'cornflowerblue')
    # Scatter points
    top_attraction_lats, top_attraction_lons= zip(*crime_lat_lon_list)
    #gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=40, marker=False)
    #gmap.heatmap(top_attraction_lats, top_attraction_lons, threshold=10, radius=40)
    #gmap.heatmap(top_attraction_lats, top_attraction_lons, threshold=10, radius=40, dissipating=False, maxIntensity=None,gradient=[(30,30,30,0), (30,30,30,1), (50, 50, 50, 1)])
    gmap.heatmap(top_attraction_lats, top_attraction_lons, threshold=100, radius=20, gradient=None, opacity=0.5, maxIntensity=4, dissipating=True)
    # Draw
    gmap.draw("last_week_crime_heatmap.html")

In [82]:
generate_past_week_html(crime_lat_lon_list)

In [96]:
def get_zipcode_crime_map(zipcode):
    from uszipcode import ZipcodeSearchEngine
    search = ZipcodeSearchEngine()
    zip_lat = search.by_zipcode(zipcode)['Latitude']
    zip_lon = search.by_zipcode(zipcode)['Longitude']
    generate_past_week_html(crime_lat_lon_list, center_lat = zip_lat, center_lon = zip_lon)
    
def get_address_crime_map(address):
    geocode_result = gmaps.geocode(address)
    if(len(geocode_result) == 0):
        print("cannot get the address")
        return
    curr_lat = geocode_result[0]["geometry"]["location"]['lat']
    curr_lon = geocode_result[0]["geometry"]["location"]['lng']
    if(check_CA_lat_lon(curr_lat, curr_lon) == False):
        print("cannot get the address")
        return
    generate_past_week_html(crime_lat_lon_list, center_lat = curr_lat, center_lon = curr_lon)

In [90]:
get_zipcode_crime_map(92139)

In [99]:
get_address_crime_map('7788 Regents Rd, San Diego, CA 92122')

In [100]:
get_address_crime_map('9500 Gilman Dr, La Jolla, CA 92093')